In [4]:
# ================================================================================
# IMPORTS - Required libraries for deep learning and data processing
# ================================================================================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, SpatialDropout2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
import json
from datetime import datetime
import numpy as np
from scipy.ndimage import gaussian_filter

In [5]:
# ================================================================================
# OUTDOOR NOISE PREPROCESSING FUNCTION
# ================================================================================
def add_outdoor_noise(image):
    """
    Simulate outdoor conditions: noise, blur, contrast variations
    This helps the model generalize to real outdoor images with:
    - Camera shake and wind (Gaussian noise)
    - Motion blur and focus issues (Gaussian blur)
    - Varying lighting conditions (contrast adjustments)
    """
    # Add Gaussian noise (wind, camera shake) - 30% chance
    if np.random.random() < 0.3:
        noise = np.random.normal(0, 0.02, image.shape)
        image = np.clip(image + noise, 0, 1)
    
    # Add slight blur (motion/focus issues) - 20% chance
    if np.random.random() < 0.2:
        sigma = np.random.uniform(0.5, 1.5)
        # Apply blur to each channel separately
        for i in range(image.shape[2]):
            image[:, :, i] = gaussian_filter(image[:, :, i], sigma=sigma)
    
    # Random contrast adjustment - 30% chance
    if np.random.random() < 0.3:
        alpha = np.random.uniform(0.8, 1.2)
        image = np.clip(alpha * image, 0, 1)
    
    return image


In [6]:
# ================================================================================
# CPU OPTIMIZATIONS - Configure TensorFlow for maximum performance
# ================================================================================
print("Configuring TensorFlow for Intel Core i5-13420H...")

# Enable Intel oneDNN optimizations (significant speedup on Intel CPUs)
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

# Configure threading for 8-core CPU (4 P-cores + 4 E-cores)
tf.config.threading.set_intra_op_parallelism_threads(8)  # Use all 8 cores
tf.config.threading.set_inter_op_parallelism_threads(2)  # Parallel operations

# Disable GPU (force CPU usage)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

print("✓ TensorFlow optimizations applied")
print(f"  • Intra-op parallelism: 8 threads")
print(f"  • Inter-op parallelism: 2 threads")
print(f"  • Intel oneDNN: Enabled")


Configuring TensorFlow for Intel Core i5-13420H...
✓ TensorFlow optimizations applied
  • Intra-op parallelism: 8 threads
  • Inter-op parallelism: 2 threads
  • Intel oneDNN: Enabled


In [7]:
# ================================================================================
# STEP 1: CONFIGURATION - Define paths and hyperparameters
# ================================================================================
# This is where we configure all the settings BEFORE training begins.
# The hyperparameters control HOW the model learns.

# Dataset paths - Where the images are stored
DATASET_PATH = r"C:\Users\altai\Desktop\DATASET\tomato leaf diseases dataset(augmented)"
TRAIN_PATH = os.path.join(DATASET_PATH, "training")     # Training images (80%)
VAL_PATH = os.path.join(DATASET_PATH, "validation")     # Validation images (20%)
MODEL_SAVE_PATH = r"C:\Users\altai\Desktop\TLDI_system\backend\trained_model_fito_outdoor.h5"
BACKUP_MODEL_PATH = r"C:\Users\altai\Desktop\TLDI_system\backend\trained_model_fito_outdoor_backup.h5"
HISTORY_PATH = r"C:\Users\altai\Desktop\TLDI_system\training_history_outdoor.json"

# HYPERPARAMETERS - Optimized for Intel Core i5-13420H + Outdoor Images
# -------------------------------------------------------------------
IMG_SIZE = 224          # Image resolution: 224x224 pixels (MobileNetV2 optimal size)
BATCH_SIZE = 32         # Number of images processed together (optimized for 8 cores)
EPOCHS = 50             # Number of complete passes through the entire dataset
LEARNING_RATE = 0.001   # Learning rate (higher for faster convergence with larger batches)
VALIDATION_SPLIT = 0.2  # 20% of data used for validation

print("=" * 80)
print("TOMATO LEAF DISEASE DETECTION - OUTDOOR + CLASS BALANCE OPTIMIZED")
print("=" * 80)
print(f"\nTraining Configuration:")
print(f"   • CPU: Intel Core i5-13420H (8 cores)")
print(f"   • RAM: 16GB")
print(f"   • Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"   • Batch Size: {BATCH_SIZE}")
print(f"   • Epochs: {EPOCHS} (with early stopping)")
print(f"   • Learning Rate: {LEARNING_RATE}")
print(f"   • Dataset Path: {DATASET_PATH}")
print(f"\nSpecial Optimizations:")
print(f"   ✓ Class weight balancing (handles imbalanced datasets)")
print(f"   ✓ Enhanced augmentation for outdoor conditions")
print(f"   ✓ Outdoor noise preprocessing (noise, blur, contrast)")
print(f"   ✓ Increased dropout for better generalization")
print(f"\nEstimated training time: 30-50 minutes")

TOMATO LEAF DISEASE DETECTION - OUTDOOR + CLASS BALANCE OPTIMIZED

Training Configuration:
   • CPU: Intel Core i5-13420H (8 cores)
   • RAM: 16GB
   • Image Size: 224x224
   • Batch Size: 32
   • Epochs: 50 (with early stopping)
   • Learning Rate: 0.001
   • Dataset Path: C:\Users\altai\Desktop\DATASET\tomato leaf diseases dataset(augmented)

Special Optimizations:
   ✓ Class weight balancing (handles imbalanced datasets)
   ✓ Enhanced augmentation for outdoor conditions
   ✓ Outdoor noise preprocessing (noise, blur, contrast)
   ✓ Increased dropout for better generalization

Estimated training time: 30-50 minutes


In [8]:
# ================================================================================
# STEP 2: DATA PREPARATION - Configure how images are loaded and augmented
# ================================================================================
# Enhanced data augmentation for OUTDOOR CONDITIONS
# This simulates real-world outdoor images with varying lighting, angles, and noise

# TRAINING DATA AUGMENTATION - ENHANCED FOR OUTDOOR CONDITIONS
print("\n" + "=" * 80)
print("STEP 2: Configuring enhanced data augmentation for outdoor conditions...")
print("=" * 80)

train_datagen = ImageDataGenerator(
    rescale=1./255,                  # Normalize pixel values from 0-255 to 0-1
    rotation_range=40,               # ↑ from 25 (more camera angles)
    width_shift_range=0.3,           # ↑ from 0.25 (off-center shots)
    height_shift_range=0.3,          # ↑ from 0.25
    shear_range=0.3,                 # ↑ from 0.25 (perspective variations)
    zoom_range=0.35,                 # ↑ from 0.25 (closer/farther shots)
    brightness_range=[0.5, 1.5],     # ↑ from [0.7, 1.3] (harsh sunlight/shadows)
    channel_shift_range=30.0,        # NEW: Color temperature variations (outdoor lighting)
    horizontal_flip=True,            # Flip images horizontally
    vertical_flip=True,              # Flip images vertically
    fill_mode='reflect',             # Changed from 'nearest' (better for outdoor backgrounds)
    preprocessing_function=add_outdoor_noise  # NEW: Simulate outdoor noise/blur
)

# VALIDATION DATA - Only normalize, NO augmentation (to test on real data)
val_datagen = ImageDataGenerator(rescale=1./255)

# LOAD TRAINING DATA - Read images from the training folder
print("\nLoading training data...")
try:
    train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,                         # Path to training images folder
        target_size=(IMG_SIZE, IMG_SIZE),   # Resize all images to 224x224
        batch_size=BATCH_SIZE,              # Load 32 images at a time
        class_mode='categorical',           # Multi-class classification (one-hot)
        shuffle=True                        # Randomize order each epoch
    )
    print(f"✓ Training data loaded successfully")
except Exception as e:
    print(f"Error loading training data: {e}")
    print(f"Please ensure training data exists at: {TRAIN_PATH}")
    raise

# LOAD VALIDATION DATA - Used to check model performance during training
print("Loading validation data...")
try:
    val_generator = val_datagen.flow_from_directory(
        VAL_PATH,                           # Path to validation images folder
        target_size=(IMG_SIZE, IMG_SIZE),   # Same size as training
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False                       # Don't shuffle for consistent evaluation
    )
    print(f"✓ Validation data loaded successfully")
except Exception as e:
    print(f"Error loading validation data: {e}")
    print(f"Please ensure validation data exists at: {VAL_PATH}")
    raise

print(f"\nDataset Summary:")
print(f"   • Training samples: {train_generator.samples}")
print(f"   • Validation samples: {val_generator.samples}")
print(f"   • Number of classes: {train_generator.num_classes}")
print(f"   • Classes: {list(train_generator.class_indices.keys())}")


STEP 2: Configuring enhanced data augmentation for outdoor conditions...

Loading training data...
Found 20452 images belonging to 11 classes.
✓ Training data loaded successfully
Loading validation data...
Found 5488 images belonging to 11 classes.
✓ Validation data loaded successfully

Dataset Summary:
   • Training samples: 20452
   • Validation samples: 5488
   • Number of classes: 11
   • Classes: ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy', 'Unidentified']


In [9]:
# ================================================================================
# STEP 3: CLASS WEIGHT CALCULATION - Handle imbalanced datasets
# ================================================================================
# Calculate class weights to ensure all classes get equal attention during training
# This prevents the model from being biased toward majority classes

print("\n" + "=" * 80)
print("STEP 3: Calculating class weights to handle imbalanced dataset...")
print("=" * 80)

# Count samples per class
class_counts = {}
for class_name in train_generator.class_indices.keys():
    class_path = os.path.join(TRAIN_PATH, class_name)
    count = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
    class_counts[class_name] = count

# Calculate class weights using inverse frequency
total_samples = train_generator.samples
num_classes = train_generator.num_classes
class_weights = {}

print("\nClass Distribution and Weights:")
print("-" * 80)
print(f"{'Class Name':<50} {'Samples':>8} {'Weight':>8} {'%':>6}")
print("-" * 80)

for class_name, class_idx in sorted(train_generator.class_indices.items(), key=lambda x: x[1]):
    count = class_counts[class_name]
    # Weight = total_samples / (num_classes * class_count)
    weight = total_samples / (num_classes * count)
    class_weights[class_idx] = weight
    percentage = (count / total_samples) * 100
    print(f"{class_name:<50} {count:>8} {weight:>8.3f} {percentage:>5.1f}%")

print("-" * 80)
print(f"{'TOTAL':<50} {total_samples:>8}")

# Calculate imbalance ratio
max_samples = max(class_counts.values())
min_samples = min(class_counts.values())
imbalance_ratio = max_samples / min_samples

print(f"\nImbalance Analysis:")
print(f"   • Maximum samples per class: {max_samples}")
print(f"   • Minimum samples per class: {min_samples}")
print(f"   • Imbalance ratio: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 2.0:
    print(f"   ⚠️  SIGNIFICANT imbalance detected - Class weights will help!")
elif imbalance_ratio > 1.5:
    print(f"   ⚠️  MODERATE imbalance detected - Class weights recommended")
else:
    print(f"Dataset is relatively balanced")


STEP 3: Calculating class weights to handle imbalanced dataset...

Class Distribution and Weights:
--------------------------------------------------------------------------------
Class Name                                          Samples   Weight      %
--------------------------------------------------------------------------------
Tomato___Bacterial_spot                                1702    1.092   8.3%
Tomato___Early_blight                                  1920    0.968   9.4%
Tomato___Late_blight                                   1851    1.004   9.1%
Tomato___Leaf_Mold                                     1882    0.988   9.2%
Tomato___Septoria_leaf_spot                            1745    1.065   8.5%
Tomato___Spider_mites Two-spotted_spider_mite          1741    1.068   8.5%
Tomato___Target_Spot                                   1827    1.018   8.9%
Tomato___Tomato_Yellow_Leaf_Curl_Virus                 1961    0.948   9.6%
Tomato___Tomato_mosaic_virus                          

In [10]:
# ================================================================================
# STEP 4: MODEL BUILDING - Create the neural network architecture
# ================================================================================
# We use TRANSFER LEARNING: start with a pre-trained model (MobileNetV2)
# that already knows how to extract image features, then add our own
# classification layers on top for tomato disease detection.

print("\n" + "=" * 80)
print("STEP 4: Building model with MobileNetV2 transfer learning...")
print("=" * 80)

# STEP 4A: Load pre-trained MobileNetV2 (trained on ImageNet - 1M+ images)
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),  # 224x224 RGB images
    include_top=False,                     # Remove original classification layer
    weights='imagenet'                     # Use pre-trained ImageNet weights
)

# STEP 4B: Freeze base model - Don't retrain the pre-trained layers
base_model.trainable = False

# STEP 4C: Add custom classification layers with INCREASED DROPOUT
# Higher dropout helps prevent overfitting to specific backgrounds
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)    # Pass through MobileNetV2
x = SpatialDropout2D(0.2)(x)              # NEW: Spatial dropout before pooling
x = GlobalAveragePooling2D()(x)           # Reduce feature maps to 1D
x = Dropout(0.4)(x)                       # ↑ from 0.3 (more robust)
x = Dense(512, activation='relu')(x)      # Fully connected layer: 512 neurons
x = Dropout(0.4)(x)                       # ↑ from 0.3 (more robust)
x = Dense(256, activation='relu')(x)      # Fully connected layer: 256 neurons
x = Dropout(0.3)(x)                       # ↑ from 0.2 (more robust)
outputs = Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer

model = Model(inputs, outputs)

# STEP 4D: Compile model - Define how it learns
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),  # Optimizer: how weights are updated
    loss='categorical_crossentropy',               # Loss function for multi-class
    metrics=['accuracy']                           # Track accuracy during training
)

print("\nModel Architecture:")
print(f"   • Base Model: MobileNetV2 (frozen)")
print(f"   • Input Shape: ({IMG_SIZE}, {IMG_SIZE}, 3)")
print(f"   • Custom Layers: SpatialDropout2D(0.2) → Dense(512) → Dense(256) → Dense({train_generator.num_classes})")
print(f"   • Dropout Rates: 0.4, 0.4, 0.3 (increased for outdoor robustness)")
print(f"   • Total Parameters: {model.count_params():,}")
print(f"   • Trainable Parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")



STEP 4: Building model with MobileNetV2 transfer learning...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 67s 7us/step

Model Architecture:
   • Base Model: MobileNetV2 (frozen)
   • Input Shape: (224, 224, 3)
   • Custom Layers: SpatialDropout2D(0.2) → Dense(512) → Dense(256) → Dense(11)
   • Dropout Rates: 0.4, 0.4, 0.3 (increased for outdoor robustness)
   • Total Parameters: 3,048,011
   • Trainable Parameters: 790,027


In [ ]:
# ================================================================================
# STEP 5: TRAINING - Train the model on the dataset WITH CLASS WEIGHTS
# ================================================================================
# This is the main training loop where the model learns from the images.
# Class weights ensure all classes get equal attention during training.

# CALLBACKS - Special functions that run during training
callbacks = [
    # ModelCheckpoint: Save the model whenever validation accuracy improves
    ModelCheckpoint(
        MODEL_SAVE_PATH,
        monitor='val_accuracy',    # Watch validation accuracy
        save_best_only=True,       # Only save if it's the best so far
        mode='max',
        verbose=1
    ),
    # EarlyStopping: Stop training if model stops improving (prevents overfitting)
    EarlyStopping(
        monitor='val_loss',
        patience=7,                # Stop if no improvement for 7 epochs
        restore_best_weights=True, # Go back to the best weights
        verbose=1
    ),
    # ReduceLROnPlateau: Lower learning rate if training plateaus
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,                # Reduce LR by half
        patience=3,                # Wait 3 epochs before reducing
        min_lr=1e-7,
        verbose=1
    )
]

# *** THIS IS WHERE THE ACTUAL TRAINING HAPPENS ***
print("\n" + "=" * 80)
print("STEP 5: Starting training with class weights and outdoor augmentation...")
print("=" * 80)
print("Training on Intel Core i5-13420H (8 cores)...")
print("Estimated time: 30-50 minutes (outdoor preprocessing adds ~20% time)\n")

# model.fit() - THE MAIN TRAINING FUNCTION WITH CLASS WEIGHTS
# Note: workers and use_multiprocessing removed for Keras 3.x compatibility
history = model.fit(
    train_generator,              # Training data
    epochs=EPOCHS,                # Train for 50 epochs (or until early stop)
    validation_data=val_generator,  # Validation data to monitor overfitting
    class_weight=class_weights,   # *** CLASS WEIGHTS FOR BALANCED TRAINING ***
    callbacks=callbacks,          # Run checkpoint, early stop, LR reduction
    verbose=1                     # Show progress bar
)



STEP 5: Starting training with class weights and outdoor augmentation...
Training on Intel Core i5-13420H (8 cores)...
Estimated time: 30-50 minutes (outdoor preprocessing adds ~20% time)

Epoch 1/50
 48/640 ━━━━━━━━━━━━━━━━━━━━ 7:53 800ms/step - accuracy: 0.3652 - loss: 1.7222

In [13]:
# ================================================================================
# STEP 6: VALIDATION/TESTING - Evaluate the trained model
# ================================================================================
# After training, we evaluate the model on the validation set one final time
# to get the official accuracy metrics.

print("\n" + "=" * 80)
print("STEP 6: Evaluating model on validation set...")
print("=" * 80)

# Evaluate the model on validation data (images it hasn't trained on)
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"\nValidation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")



STEP 6: Evaluating model on validation set...
 27/172 ━━━━━━━━━━━━━━━━━━━━ 49s 341ms/step - accuracy: 0.7812 - loss: 0.7390

KeyboardInterrupt: 

In [ ]:
# ================================================================================
# STEP 7: SAVING - Save the trained model and training history
# ================================================================================
# Save the model file (.h5) and training metrics for later analysis.

print(f"\nSTEP 7: Saving training history...")
history_dict = {
    'accuracy': [float(x) for x in history.history['accuracy']],
    'loss': [float(x) for x in history.history['loss']],
    'val_accuracy': [float(x) for x in history.history['val_accuracy']],
    'val_loss': [float(x) for x in history.history['val_loss']],
    'final_val_accuracy': float(val_accuracy),
    'final_val_loss': float(val_loss),
    'timestamp': datetime.now().isoformat(),
    'config': {
        'cpu': 'Intel Core i5-13420H (8 cores)',
        'ram': '16GB',
        'img_size': IMG_SIZE,
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS,
        'learning_rate': LEARNING_RATE,
        'training_samples': train_generator.samples,
        'validation_samples': val_generator.samples,
        'num_classes': train_generator.num_classes,
        'classes': list(train_generator.class_indices.keys()),
        'class_weights': {str(k): float(v) for k, v in class_weights.items()},
        'imbalance_ratio': float(imbalance_ratio),
        'optimizations': [
            'class_weight_balancing',
            'outdoor_augmentation',
            'outdoor_noise_preprocessing',
            'increased_dropout',
            'spatial_dropout'
        ]
    }
}

with open(HISTORY_PATH, 'w') as f:
    json.dump(history_dict, f, indent=2)
print(f"History saved to: {HISTORY_PATH}")

print("\n" + "=" * 80)
print("TRAINING COMPLETE!")
print("=" * 80)
print(f"\nModel saved at: {MODEL_SAVE_PATH}")
print(f"\nFinal Training Results:")
print(f"   • Validation Accuracy: {val_accuracy:.2%}")
print(f"   • Validation Loss: {val_loss:.4f}")
print(f"   • Total Epochs Trained: {len(history.history['accuracy'])}")
print(f"   • Best Epoch: {history.history['val_accuracy'].index(max(history.history['val_accuracy'])) + 1}")

print(f"\nNEXT STEPS:")
print(f"   1. Test the model with OUTDOOR tomato leaf images")
print(f"   2. Compare performance with previous model (trained_model_fito.h5)")
print(f"   3. Update backend to use new model:")
print(f"      - Rename: trained_model_fito_outdoor.h5 → trained_model_fito.h5")
print(f"   4. Review training history at: {HISTORY_PATH}")

print(f"\nOptimizations Applied:")
print(f"   ✓ Class weight balancing (imbalance ratio: {imbalance_ratio:.2f}:1)")
print(f"   ✓ Enhanced outdoor augmentation (rotation, zoom, brightness)")
print(f"   ✓ Outdoor noise preprocessing (noise, blur, contrast)")
print(f"   ✓ Increased dropout (0.4, 0.4, 0.3) for robustness")
print(f"   ✓ Spatial dropout before pooling")
print(f"   ✓ Intel oneDNN + 8-core parallelization")

print(f"\nExpected Improvements:")
print(f"   • 30-50% better accuracy on outdoor images")
print(f"   • Balanced performance across all disease classes")
print(f"   • Better handling of harsh lighting and shadows")
print(f"   • More robust to different backgrounds and angles")
print(f"   • Reduced bias toward majority classes")

print("\n" + "=" * 80)
